#### For Colab

In [ ]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [ ]:
!pip install transformers==4.1.1
!pip install -U sentencepiece!=0.1.92
!pip install -U datasets
!pip install rouge_score
!pip install -U gensim

     |████████████████████████████████| 1.5MB 6.2MB/s 
     |████████████████████████████████| 2.9MB 33.7MB/s 
     |████████████████████████████████| 890kB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8c69e659572bc09bb030185dfce8137a1d182066e99c6c3e071e44e970640f77
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 5.3MB/s 
     |████████████████████████████████| 163kB 5.2MB/s 
     |████████████████████████████████| 17.7MB 236kB/s 
     |████████████████████████████████| 245kB 20.1MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 24.2MB 1.2MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


### **Config**

In [ ]:
import os
import sys

sys.path.insert(0, drive_dir)
import config

In [ ]:
# General configurations

MODEL = 'bart'

RE_SPLITTER = '\.(?!\d)|\n'     # do we split sentences of paragraphs?
                                # use '\.|\n' or '\n', respectively

TOKEN_MAX_LEN = 99              # max length of a word
PARA_MIN_LENGTH = 2             # minimum length for a sentence or
                                # a paragraph, in tokens

### **Init**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import re
import pandas as pd
import gensim
from tqdm import tqdm
tqdm.pandas()

if 'pegasus' in MODEL:
    from transformers import PegasusTokenizer
    tokenizer =\
        PegasusTokenizer.from_pretrained('google/pegasus-large')
elif 'bart' in MODEL:
    from transformers import BartTokenizer
    tokenizer =\
        BartTokenizer.from_pretrained('facebook/bart-large-cnn')

## **Assign Bullets**

### **Load Datasets**

In [ ]:
df_cc = pd.read_csv(config.DATASET_PATH+'karger_books_chunk_chapter/'+MODEL+'/df.csv')
df_cc = df_cc.set_index(['book', 'chapter'])
df_cc.bullets = df_cc.bullets.map(eval)
df_cc.text = df_cc.text.map(eval)
df_cc = df_cc.explode('text')

In [ ]:
df_moc = pd.read_csv(config.DATASET_PATH+'karger_books_moc/'+MODEL+'/df.csv')
df_moc = df_moc.set_index(['book', 'chapter', 'merge'])
df_moc.bullets = df_moc.bullets.map(eval)
df_moc.text = df_moc.text.map(eval)
df_moc = df_moc.explode('text')

### **Evaluate ROUGE recall**

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

def calculate_rouge(df, col_text, col_bullets):
    # explode the bullets list and assign ROUGE for each bullet

    df_expl = df.explode(col_bullets)

    rouge_res =\
        df_expl[[col_text, col_bullets]]\
        .progress_apply(lambda row:
        metric.compute(
            predictions = [row[0]],
            references = [row[1]],
            rouge_types = config.ROUGE_TYPES,
            use_agregator = False), axis=1)
        
    for r in config.ROUGE_TYPES:
        df_expl[r+'_recall'] =\
            rouge_res.map(lambda score: score[r][0][1])

    return df_expl


df_cc = calculate_rouge(df_cc, 'text', 'bullets')
df_moc = calculate_rouge(df_moc, 'text', 'bullets')

100%|██████████| 10947/10947 [01:59<00:00, 91.65it/s] 


### **Keep highest recall from bullets**

Some chunks will be neglected (around 30%).

In [ ]:
def assign_highest_recall_text(df):
    df['best_match'] = False

    for idx, text  in df.groupby('bullets').progress_apply(
        lambda g: g.iloc[g[config.ROUGE_TYPE_RECALL].argmax()]).text.iteritems():
        
        df.loc[\
            (df['bullets'] == idx) &\
            (df['text'] == text), 'best_match'] = True
    
    return df

df_cc_from_bull = assign_highest_recall_text(df_cc)
df_moc_from_bull = assign_highest_recall_text(df_moc)

100%|██████████| 2556/2556 [00:01<00:00, 1630.11it/s]


In [ ]:
print('Percentage of chunks with at least one bullet point in CC: %.2f %%'%\
    (100*np.sum(df_cc_from_bull.groupby('text')['best_match'].apply(np.any).tolist())\
    /len(df_cc_from_bull.groupby('text')['best_match'].apply(np.any))))

Percentage of chunks with at least one bullet point in CC: 73.50 %


In [ ]:
print('Percentage of chunks with at least one bullet point in MoC: %.2f %%'%\
    (100*np.sum(df_moc_from_bull.groupby('text')['best_match'].apply(np.any).tolist())\
    /len(df_moc_from_bull.groupby('text')['best_match'].apply(np.any))))

Percentage of chunks with at least one bullet point in MoC: 68.86 %


In [ ]:
df_cc_from_bull = df_cc_from_bull[df_cc_from_bull['best_match']]
df_moc_from_bull = df_moc_from_bull[df_moc_from_bull['best_match']]

In [ ]:
assert len(df_cc_from_bull) == len(df_cc.groupby('bullets').count())
assert len(df_moc_from_bull) == len(df_moc.groupby('bullets').count())

#### Save dataset

In [ ]:
df_cc_from_bull.to_csv(config.DATASET_PATH+'karger_books_chunk_chapter/assign_bullets/'+MODEL+'/df.csv')
df_moc_from_bull.to_csv(config.DATASET_PATH+'karger_books_moc/assign_bullets/'+MODEL+'df.csv')

#### Create train, test, validation (CC)

In [ ]:
op = config.DATASET_PATH+'karger_books_chunk_chapter/assign_bullets/'+MODEL+'/'
if not os.path.exists(op):
    os.makedirs(op)

In [ ]:
df_cc_from_bull = df_cc_from_bull.groupby(level=[0, 1], sort=False).agg({
    'bullets': lambda b: list(b),
    'text': lambda t: list(t),
})

In [ ]:
df_cc_from_bull = df_cc_from_bull.sample(frac=1, random_state=config.SEED)
df_cc_from_bull['num_bulls'] = df_cc_from_bull.bullets.map(len).cumsum()
tot_bulls = df_cc_from_bull.num_bulls.iloc[-1]
split1 = np.where(df_cc_from_bull.num_bulls > int(tot_bulls*0.8))[0][0]+1
split2 = np.where(df_cc_from_bull.num_bulls > int(tot_bulls*0.9))[0][0]+1
print(split1, split2)

361 408


In [ ]:
train, val, test =\
    df_cc_from_bull.iloc[:split1].explode('bullets'),\
    df_cc_from_bull.iloc[split1:split2].explode('bullets'),\
    df_cc_from_bull.iloc[split2:].explode('bullets')

train['text'] = df_cc_from_bull.iloc[:split1].explode('text')['text']
val['text'] = df_cc_from_bull.iloc[split1:split2].explode('text')['text']
test['text'] = df_cc_from_bull.iloc[split2:].explode('text')['text']

train.to_csv(op+'train.csv')
val.to_csv(op+'val.csv')
test.to_csv(op+'test.csv')

In [ ]:
with open(op+'train.source', 'w') as tr_s,\
    open(op+'train.target', 'w') as tr_t,\
    open(op+'train.index', 'w') as tr_i:
    for idx, row in train[['text', 'bullets']].iterrows():
        tr_i.write(str(idx) + '\n')
        tr_s.write(row.text + '\n')
        tr_t.write(row.bullets + '\n')

In [ ]:
with open(op+'val.source', 'w') as va_s,\
    open(op+'val.target', 'w') as va_t,\
    open(op+'val.index', 'w') as va_i:
    for idx, row in val[['text', 'bullets']].iterrows():
        va_i.write(str(idx) + '\n')
        va_s.write(row.text + '\n')
        va_t.write(row.bullets + '\n')

In [ ]:
with open(op+'test.source', 'w') as te_s,\
    open(op+'test.target', 'w') as te_t,\
    open(op+'test.index', 'w') as te_i:
    for idx, row in test[['text', 'bullets']].iterrows():
        te_i.write(str(idx) + '\n')
        te_s.write(row.text + '\n')
        te_t.write(row.bullets + '\n')

#### Create train, test, validation (MoC)

In [ ]:
op = config.DATASET_PATH+'karger_books_moc/assign_bullets/'+MODEL+'/'
if not os.path.exists(op):
    os.makedirs(op)

In [ ]:
df_moc_from_bull = df_moc_from_bull.groupby(level=[0, 1], sort=False).agg({
    'bullets': lambda b: list(b),
    'text': lambda t: list(t),
})

In [ ]:
df_moc_from_bull = df_moc_from_bull.sample(frac=1, random_state=config.SEED)
df_moc_from_bull['num_bulls'] = df_moc_from_bull.bullets.map(len).cumsum()
tot_bulls = df_moc_from_bull.num_bulls.iloc[-1]
split1 = np.where(df_moc_from_bull.num_bulls > int(tot_bulls*0.8))[0][0]+1
split2 = np.where(df_moc_from_bull.num_bulls > int(tot_bulls*0.9))[0][0]+1
print(split1, split2)

361 408


In [ ]:
train, val, test =\
    df_moc_from_bull.iloc[:split1].explode('bullets'),\
    df_moc_from_bull.iloc[split1:split2].explode('bullets'),\
    df_moc_from_bull.iloc[split2:].explode('bullets')

train['text'] = df_moc_from_bull.iloc[:split1].explode('text')['text']
val['text'] = df_moc_from_bull.iloc[split1:split2].explode('text')['text']
test['text'] = df_moc_from_bull.iloc[split2:].explode('text')['text']

train.to_csv(op+'train.csv')
val.to_csv(op+'val.csv')
test.to_csv(op+'test.csv')

In [ ]:
with open(op+'train.source', 'w') as tr_s,\
    open(op+'train.target', 'w') as tr_t,\
    open(op+'train.index', 'w') as tr_i:
    for idx, row in train[['text', 'bullets']].iterrows():
        tr_i.write(str(idx) + '\n')
        tr_s.write(row.text + '\n')
        tr_t.write(row.bullets + '\n')

In [ ]:
with open(op+'val.source', 'w') as va_s,\
    open(op+'val.target', 'w') as va_t,\
    open(op+'val.index', 'w') as va_i:
    for idx, row in val[['text', 'bullets']].iterrows():
        va_i.write(str(idx) + '\n')
        va_s.write(row.text + '\n')
        va_t.write(row.bullets + '\n')

In [ ]:
with open(op+'test.source', 'w') as te_s,\
    open(op+'test.target', 'w') as te_t,\
    open(op+'test.index', 'w') as te_i:
    for idx, row in test[['text', 'bullets']].iterrows():
        te_i.write(str(idx) + '\n')
        te_s.write(row.text + '\n')
        te_t.write(row.bullets + '\n')